# Growing classes

When implementing much of the functionality and running the research whose artifacts live in this repository, the authors found it best to document the iterations of the research and development. However, Python insists classes should be defined in one block, complicating the iterative development of its methods. We thus write here a decorator that allows for the definition of classes one method at a time, across multiple code cells.

In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120 --ignore W293,E302

In [2]:
from contextlib import contextmanager
from dask.delayed import Delayed
import dask
from functools import reduce
import inspect
from jupytest import Suite, Report, summarize_results
import re
from typing import Callable, Sequence, Optional, cast, Set, Union

In [3]:
suite = Suite()
if __name__ == "__main__":
    suite |= Report()

In [4]:
Decorator = Callable[[Callable], Callable]


def growing(klass: type) -> type:
    def add_method(
        fn_: Optional[Callable] = None,
        name: str = "",
        wrapped_in: Union[Decorator, Sequence[Decorator]] = []
    ) -> Callable:
        def add_to_class(fn: Callable):
            name_method = name or fn.__name__
            method_new = reduce(lambda f, w: w(f), wrapped_in if hasattr(wrapped_in, "__iter__") else [wrapped_in], fn)
            setattr(klass, name_method, method_new)
            return getattr(klass, name_method)
        
        if fn_ is None:
            return add_to_class
        return add_to_class(cast(Callable, fn_))
    
    def add_class_method(
        fn_: Optional[Callable] = None,
        name: str = "",
        wrapped_in: Union[Decorator, Sequence[Decorator]] = []
    ) -> Callable:
        wrappers = wrapped_in if hasattr(wrapped_in, "__iter__") else [wrapped_in]
        return add_method(fn_, name, wrappers + [classmethod])

    setattr(klass, "method", staticmethod(add_method))
    setattr(klass, "classmethod", staticmethod(add_class_method))
    return klass

## Tests

In [5]:
def user_members(klass) -> Set[str]:
    return {m for m in dir(klass) if not re.match(r"^__.*__$", m)}

In [6]:
%%test Add method
@growing
class MyClass:
    def f(self):
        return 5

assert {"f", "method"} <= user_members(MyClass)
assert "g" not in user_members(MyClass)


@MyClass.method
def g(self, x):
    return self.f() + x

assert {"f", "g", "method"} <= user_members(MyClass)
assert MyClass().g(3) == 8

Test Add method passed.


In [7]:
%%test Add Dask Delayed method
@growing
class MyClass:
    def f(self):
        return 5

@MyClass.method(wrapped_in=dask.delayed(pure=True))
def h(self, x, y):
    return self.f() * x + y

assert "h" in user_members(MyClass)
assert isinstance(MyClass().h(4, 5), Delayed)
assert MyClass().h(4, 5).compute(scheduler="single-threaded") == 25

Test Add Dask Delayed method passed.


In [8]:
%%test Multiple method wrappers
@growing
class MyClass:
    def f(self):
        return 5

def wrapper1(fn):
    return lambda self, x: fn(self, x) + x

def wrapper2(fn):
    return lambda self, x: fn(self, x) * x

@MyClass.method(wrapped_in=[wrapper1, wrapper2])
def double_wrapped(self, x):
    return x / 3 + self.f()
    
assert "double_wrapped" in user_members(MyClass)
assert MyClass().double_wrapped(9) == 153.0

Test Multiple method wrappers passed.


In [9]:
%%test Add class method, inelegant
@growing
class MyClass:
    C = 34
    
    def f(self):
        return 5

try:
    @MyClass.method
    @classmethod
    def cm(cls):
        return cls.C
    fail()
except AttributeError:
    pass

@MyClass.method(wrapped_in=classmethod)
def cm(cls):
    return cls.C
assert MyClass.cm() == MyClass.C

Test Add class method, inelegant passed.


In [10]:
%%test Add class method, preferred approach
@growing
class MyClass:
    C = 34
    
    def f(self):
        return 5

@MyClass.classmethod
def cm(cls):
    return cls.C


assert MyClass.cm() == MyClass.C

Test Add class method, preferred approach passed.


In [11]:
%%test Add class method that acts as context manager
@growing
class MyClass:
    C = 34
    def f(self):
        return 5
    
    
@MyClass.classmethod(wrapped_in=contextmanager)
def changing_C(cls, num: int):
    old = cls.C
    try:
        cls.C = num
        yield
    finally:
        cls.C = old
        
        
assert MyClass.C == 34
with MyClass.changing_C(45):
    assert MyClass.C == 45
assert MyClass.C == 34

Test Add class method that acts as context manager passed.


In [12]:
if __name__ == "__main__":
    _ = summarize_results(suite)

6 passed, 0 failed, 0 raised an error
